# Entrada de Estacionamento de Shopping Center

Em um “Shopping Center” chegam ao estacionamento 5 veículos por minuto, em horário de almoço. Existe um único bloqueio de entrada e os veículos que chegam entram em uma única fila.

Cada motorista, ao chegar sua vez, gasta 10 segundos entre avançar o veículo até o controle do bloqueio, pressionar o botão, retirar o ticket de entrada, esperar a cancela ser aberta e seguir em frente com o veículo.

Perguntas:
1. Qual é o tamanho médio da fila de veículos?
2. Qual é o tempo médio que um motorista gasta desde que chega à fila até entrar no estacionamento?
3. Quantos bloqueios devem ser colocados para melhorar o atendimento?

In [1]:
!pip3 install simpy

In [0]:
import itertools
import numpy as np

import simpy
import pandas as pd

In [7]:
numero_de_carros = 1000
numero_de_bloqueios = 3

esperas = []
def carro(env, bloqueio):
    with bloqueio.request() as req:
        chegada = env.now
        yield req
        espera = env.now - chegada
        esperas.append(espera)
        yield env.timeout(20)

def hora_do_almoco(env, numero_de_carros, bloqueio):
    for i in range(numero_de_carros):
        env.process(carro(env, bloqueio))
        yield env.timeout(3600 / numero_de_carros)

env = simpy.Environment()
bloqueio = simpy.Resource(env, capacity=numero_de_bloqueios)
env.process(hora_do_almoco(env, numero_de_carros, bloqueio))
env.run(until=3600)

pd.Series(esperas).divide(60)\
                  .describe()\
                  .rename('Tempo para entrar em minutos')\
                  .to_frame()

,Tempo para entrar em minutos
count,540.000000
mean,13.723333
std,7.974698
min,0.000000
25%,6.861667
50%,13.723333
75%,20.585000
max,27.446667
